In [ ]:
#!/usr/bin/env python

import itertools

import numpy
import pandas
import pickle
import random
import scipy.stats

from matplotlib import pyplot

%matplotlib inline

In [ ]:
mut_effect_dict = pickle.load(open('../31_fitness/mut_effect_dict.pkl', 'rb'))
mut_all_list = list(mut_effect_dict.keys())
gt_mut_list = pickle.load(open('../31_fitness/gt_mut_list.pkl', 'rb'))
gt_fit_list = pickle.load(open('../31_fitness/gt_fit_list.pkl', 'rb'))
gt_fitrep_list = pickle.load(open('../31_fitness/gt_fitrep_list.pkl', 'rb'))
gt_graph = pickle.load(open('../31_fitness/gt_graph.pkl', 'rb'))
gt_dict = pickle.load(open('../31_fitness/gt_dict.pkl', 'rb'))
wt = open('../00_data/ExpData.txt').readlines()[1].split()[3]

In [ ]:
# fig 1a example of idiosyncratic mutational effects of single mutations
fs1, fs2, alpha = 16, 14, 0.6
all_effects_list = []
gt_fit_pool = numpy.array([x for x in gt_fit_list if x > 1.0])
bins = 30
loc = (1, 2, 1)
numpy.random.seed(4033)
for i, mut in enumerate([(10, 'G', 'A'),]):
    fig, ax = pyplot.subplots(1, 1, figsize=(6, 6))
    mut_effects = []
    for x in mut_effect_dict[mut]:
        if x[2] == 1.0 or x[3] == 1.0:
            continue
        mut_effects.append(numpy.log(x[3] / x[2]))
    mut_effects = numpy.array(mut_effects)
    n = mut_effects.shape[0]
    mrange = mut_effects.max() - mut_effects.min()
    print(n)
    print(mut_effects.max(), mut_effects.min())
    mstddev = numpy.std(mut_effects)
    null_effects = numpy.array([numpy.log(x[1] / x[0]) for x in numpy.random.choice(gt_fit_pool, size=(n,2), replace=True)])
    null_range = null_effects.max() - null_effects.min()
    print(null_effects.max(), null_effects.min())
    null_stddev = numpy.std(null_effects)
    print(f'mrange:{mrange:.3f}\tnull_range:{null_range:.3f}\tmrange / null_range:{mrange / null_range:.3f}')
    print(f'mstddev:{mstddev:.3f}\tnull_stddev:{null_stddev:.3f}\tmstddev / null_stddev:{mstddev / null_stddev:.3f}')
    xmin = min(mut_effects.min(), null_effects.min())
    xmax = max(mut_effects.max(), null_effects.max())
    bins = numpy.arange(xmin, xmax+0.001, 0.05)
    ax.hist(mut_effects,bins=bins, color='darkorchid', alpha=alpha, histtype='bar', linewidth=2)
    ax.hist(null_effects,bins=bins, color='grey', alpha=alpha, histtype='bar', linewidth=2, zorder=-1)
    ax.set_xlabel(f'Fitness effects of the {mut[1]}-to-{mut[2]}\nmutation at position 10', fontsize=fs1, labelpad=10)
    ax.set_ylabel('Frequency', fontsize=fs1)
    ax.legend(labels=('Observed', 'Random'), fontsize=fs1, loc=loc[i], frameon=False)
    [x.set_fontsize(fs2) for x in ax.xaxis.get_ticklabels()]
    [x.set_fontsize(fs2) for x in ax.yaxis.get_ticklabels()]
    fig.tight_layout()
    fig.savefig(f'hist_effect_mut{mut[1]}{mut[0]}{mut[2]}.pdf', dpi=300)

In [ ]:
# distribution of Iid values of all single mutations
fs1, fs2, alpha = 16, 14, 0.6
ns, ranges, stddevs, effects_list, mutations, all_effects_list = [], [], [], [], [], []
for pos in range(1, 70):
#     print(pos, end=' ')
    for nt1, nt2 in itertools.permutations('ACGT', 2):
        mut = (pos, nt1, nt2)
        mut_effects = []
        for x in mut_effect_dict[mut]:
            if x[2] == 1.0 or x[3] == 1.0:
                continue
            mut_effects.append(numpy.log(x[3] / x[2]))
        all_effects_list.extend(mut_effects)
        mut_effects = numpy.array(mut_effects)
        ns.append(mut_effects.shape[0])
        mutations.append(mut)
        ranges.append(mut_effects.max() - mut_effects.min())
        stddevs.append(numpy.std(mut_effects))
        effects_list.append(mut_effects)
all_effects_arr = numpy.array(all_effects_list)

gt_fit_pool = numpy.array([x for x in gt_fit_list if x > 1.0])
null_ranges, null_stddevs, null_effects_list, single_ranges, single_stddevs, single_effects_list = [], [], [], [], [], []
for n in ns:
    numpy.random.seed(n**2 + 3)
    null_effects = numpy.array([numpy.log(x[1] / x[0]) for x in numpy.random.choice(gt_fit_pool, size=(n,2), replace=True)])
    null_ranges.append(null_effects.max() - null_effects.min())
    null_stddevs.append(numpy.std(null_effects))
    null_effects_list.append(null_effects)
print(len(ranges))
ranges = numpy.array(ranges)
null_ranges = numpy.array(null_ranges)
single_ranges = numpy.array(single_ranges)
stddevs = numpy.array(stddevs)
null_stddevs = numpy.array(null_stddevs)
single_stddevs = numpy.array(single_stddevs)

range_ratio_null = ranges / null_ranges
stddevs_ratio_null = stddevs / null_stddevs
print(f'range ratio (obs / null) mean = {numpy.mean(range_ratio_null):.5f}, '
      f'median = {numpy.median(range_ratio_null):.5f}, '
      f'std. dev. = {numpy.std(range_ratio_null):.5f}, '
      f'std. err. = {numpy.std(range_ratio_null)/numpy.sqrt(range_ratio_null.shape[0]):.5f}, '
      f'{numpy.mean(range_ratio_null)+numpy.std(range_ratio_null)/numpy.sqrt(range_ratio_null.shape[0]):.5f}, '
      f'{numpy.mean(range_ratio_null)-numpy.std(range_ratio_null)/numpy.sqrt(range_ratio_null.shape[0]):.5f}')
print(f'stddevs ratio (obs / null) mean = {numpy.mean(stddevs_ratio_null):.5f}, '
      f'median = {numpy.median(stddevs_ratio_null):.5f}, '
      f'std. dev. = {numpy.std(stddevs_ratio_null):.5f}, '
      f'std. err. = {numpy.std(stddevs_ratio_null)/numpy.sqrt(stddevs_ratio_null.shape[0]):.5f}, '
      f'{numpy.mean(stddevs_ratio_null)+numpy.std(stddevs_ratio_null)/numpy.sqrt(stddevs_ratio_null.shape[0]):.5f}, '
      f'{numpy.mean(stddevs_ratio_null)-numpy.std(stddevs_ratio_null)/numpy.sqrt(stddevs_ratio_null.shape[0]):.5f}')
print('='*120)
fig1, axs1 = pyplot.subplots(figsize=(6, 6))
xmin = range_ratio_null.min()
xmax = range_ratio_null.max()
bins = numpy.arange(xmin, xmax+0.001, 0.05)
axs1.hist(range_ratio_null,bins=bins, color='royalblue', alpha=1, histtype='bar', linewidth=2)
axs1.set_xlabel(r'$I_{id}$ of individual mutations', fontsize=fs1)
axs1.set_ylabel('Frequency', fontsize=fs1)
[x.set_fontsize(fs2) for x in axs1.xaxis.get_ticklabels()]
[x.set_fontsize(fs2) for x in axs1.yaxis.get_ticklabels()]
fig1.tight_layout()
fig1.savefig('hist_iid_ranges.pdf', dpi=300)
fig1, axs1 = pyplot.subplots(figsize=(6, 6))
xmin = stddevs_ratio_null.min()
xmax = stddevs_ratio_null.max()
bins = numpy.arange(xmin, xmax+0.001, 0.05)
axs1.hist(stddevs_ratio_null,bins=bins, color='royalblue', alpha=1, histtype='bar', linewidth=2)
axs1.set_xlabel(r'$I_{id}$ of individual mutations', fontsize=fs1)
axs1.set_ylabel('Frequency', fontsize=fs1)
[x.set_fontsize(fs2) for x in axs1.xaxis.get_ticklabels()]
[x.set_fontsize(fs2) for x in axs1.yaxis.get_ticklabels()]
fig1.tight_layout()
fig1.savefig('hist_iid_stddevs.pdf', dpi=300)

In [ ]:
# Iid distribution using different number of replicates
iid_list = []
for nrep in range(1, 7):
    ns, ranges, stddevs, effects_list, mutations, all_effects_list = [], [], [], [], [], []
    for pos in range(1, 70):
        for nt1, nt2 in itertools.permutations('ACGT', 2):
            mut = (pos, nt1, nt2)
            mut_effects = []
            for x in mut_effect_dict[mut]:
                mean_start = numpy.mean(x[4][:nrep])
                mean_end = numpy.mean(x[5][:nrep])
                if mean_start == 1.0 or mean_end == 1.0:
                    continue
                mut_effects.append(numpy.log(mean_end / mean_start))
            all_effects_list.extend(mut_effects)
            mut_effects = numpy.array(mut_effects)
            ns.append(mut_effects.shape[0])
            mutations.append(mut)
            ranges.append(mut_effects.max() - mut_effects.min())
            stddevs.append(numpy.std(mut_effects))
            effects_list.append(mut_effects)
    all_effects_arr = numpy.array(all_effects_list)

    gt_fit_pool = numpy.array([numpy.mean(x[:nrep]) for x in gt_fitrep_list if numpy.mean(x[:nrep]) > 1.0])
    null_ranges, null_stddevs, null_effects_list, single_ranges, single_stddevs, single_effects_list = [], [], [], [], [], []
    for n in ns:
        numpy.random.seed(n**2 + 3)
        null_effects = numpy.array([numpy.log(x[1] / x[0]) for x in numpy.random.choice(gt_fit_pool, size=(n,2), replace=True)])
        null_ranges.append(null_effects.max() - null_effects.min())
        null_stddevs.append(numpy.std(null_effects))
        null_effects_list.append(null_effects)
    print(nrep, len(ranges))
    ranges = numpy.array(ranges)
    null_ranges = numpy.array(null_ranges)
    stddevs = numpy.array(stddevs)
    null_stddevs = numpy.array(null_stddevs)
    range_ratio_null = ranges / null_ranges
    stddevs_ratio_null = stddevs / null_stddevs
    print(f'range ratio (obs / null) mean = {numpy.nanmean(range_ratio_null):.5f}, '
          f'median = {numpy.nanmedian(range_ratio_null):.5f}, '
          f'std. dev. = {numpy.nanstd(range_ratio_null):.5f}, '
          f'std. err. = {numpy.nanstd(range_ratio_null)/numpy.sqrt(range_ratio_null.shape[0]):.5f}')
    print(f'stddevs ratio (obs / null) mean = {numpy.nanmean(stddevs_ratio_null):.5f}, '
          f'median = {numpy.nanmedian(stddevs_ratio_null):.5f}, '
          f'std. dev. = {numpy.nanstd(stddevs_ratio_null):.5f}')
    print('='*120)
    iid_list.append(stddevs_ratio_null[numpy.isfinite(stddevs_ratio_null)])

In [ ]:
fig1, ax1 = pyplot.subplots(1, 1, figsize=(6, 6))
print(len(iid_list))
props = {'color':'black', 'linewidth':1.5}
meanprops = {'marker':'o', 'markerfacecolor':'darkorchid', 'markeredgecolor':'darkorchid'}
flierprops = {'markeredgecolor':'dimgrey', 'marker':'.', 'markerfacecolor':'dimgrey'}
ax1.boxplot(iid_list, widths=0.55, showmeans=True, medianprops=props, meanprops=meanprops,
            boxprops=props, capprops=props, whiskerprops=props, flierprops=flierprops)
ax1.set_ylim(-0.1, 2.0)
[x.set_fontsize(fs2) for x in ax1.xaxis.get_ticklabels()]
[x.set_fontsize(fs2) for x in ax1.yaxis.get_ticklabels()]
ax1.set_xlabel('Number of replicates used\nin fitness estimation', fontsize=fs1)
ax1.set_ylabel(r'$I_{id}$ of individual mutations', fontsize=fs1, rotation=90)
fig1.tight_layout()
fig1.savefig('boxplot_iidrep_stddevs.pdf', dpi=300)